This script applies filtering and ICA to KIT-NYU MEG data and makes epochs + source models of brain activity using dSPM. It expects that you have already:
- applied CALM noise reduction using the KIT-NYU MEG160 application
- converted KIT-NYU .sqd recordings into more compatible .fif files using the head shape and marker files from the session.
- coregistered the .fif file with 'fsaverage' for making source models of brain activity

In [ ]:
# Imports and setup
import mne
import os
import numpy as np
import pandas as pd
from os.path import join
import matplotlib.pyplot as plt

mne.set_log_level(verbose='WARNING')

In [ ]:
# Configuration
EXPERIMENT_NAME = 'experiment'
ROOT_DIR = 'data'
RAW_DIR = join(ROOT_DIR, 'raw')
MEG_DIR = join(ROOT_DIR, 'meg')
LOG_DIR = join(ROOT_DIR, 'logs')
STC_DIR = join(ROOT_DIR, 'stc')
SUBJECTS_DIR = join(ROOT_DIR, 'mri')

EXCLUDED_SUBJECTS = ['subject_000']
subjects = [f'subject_{i:03d}' for i in range(1, 21)]
subjects = [s for s in subjects if s not in EXCLUDED_SUBJECTS]

print(subjects)
print(f'N = {len(subjects)}')

In [ ]:
# Data loading functions
def load_meg_data(subject_id, data_type='raw'):
   subject_dir = os.path.join(MEG_DIR, subject_id)
   
   if data_type == 'raw':
       fname = os.path.join(subject_dir, f'{subject_id}_{EXPERIMENT_NAME}-raw.fif')
   elif data_type == 'ica':
       fname = os.path.join(subject_dir, f'{subject_id}_{EXPERIMENT_NAME}-ica-raw.fif')
   elif data_type == 'epochs':
       fname = os.path.join(subject_dir, f'{subject_id}_{EXPERIMENT_NAME}-ica-epo.fif')
       return mne.read_epochs(fname)
   else:
       raise ValueError("data_type must be 'raw', 'ica', or 'epochs'")
   
   return mne.io.read_raw_fif(fname, preload=True)

def load_log_file(subject_id):
   log_filename = os.path.join(LOG_DIR, subject_id, f'{subject_id}_{EXPERIMENT_NAME}_logfile.csv')
   return pd.read_csv(log_filename)

In [ ]:
# Load and filter data
subject_id = subjects[0] # Do this by subject
raw = load_meg_data(subject_id, 'raw')
events = mne.find_events(raw, min_duration=0.002)

print(f'Events found: {len(events)}')
print(f'Sampling rate: {raw.info["sfreq"]} Hz')

# Apply bandpass filter
print('Applying bandpass filter (1-40 Hz)...')
raw.filter(1, 40, method='iir')

In [ ]:
# Bad channel interpolation
bad_channels = raw.info['bads']
print(f'Bad channels: {bad_channels}')

if bad_channels:
    print('Interpolating bad channels...')
    raw.interpolate_bads()

In [ ]:
# ICA setup and fitting
ica = mne.preprocessing.ICA(
    n_components=0.95, 
    method='fastica', 
    random_state=42
)

print('Fitting ICA...')
ica.fit(raw)

In [ ]:
# ICA component visualization
ica.plot_sources(raw)
ica.plot_components()

In [ ]:
# Apply ICA
print(f'Excluding ICA components: {ica.exclude}')
raw_clean = ica.apply(raw.copy())
output_fname = join(MEG_DIR, subject_id, f'{subject_id}_{EXPERIMENT_NAME}-ica-raw.fif')
raw_clean.save(output_fname, overwrite=True)

In [ ]:
# Cell 9 - Epoching parameters
EVENT_MAPPING = {
    'condition_1': 160,
    'condition_2': 161,
    'condition_3': 162,
    'condition_4': 163,
}

EPOCH_TMIN = -0.9
EPOCH_TMAX = 0.8
BASELINE = (-0.9, -0.8)
DECIM = 1
ONSET_DELAY = 60
REJECT_THRESHOLD = {'mag': 3e-12}

In [ ]:
# Epoching function
def make_epochs(subject_id):
    print(f'Processing subject: {subject_id}')
    
    raw = load_meg_data(subject_id, 'ica')
    log = load_log_file(subject_id)
    
    picks_meg = mne.pick_types(raw.info, meg=True, eeg=False, eog=False, stim=False)
    
    events = mne.find_events(raw, min_duration=0.002)
    events[:, 0] += ONSET_DELAY
    
    epochs = mne.Epochs(
        raw, events, event_id=EVENT_MAPPING,
        tmin=EPOCH_TMIN, tmax=EPOCH_TMAX,
        baseline=BASELINE, picks=picks_meg,
        metadata=log, decim=DECIM, preload=True
    )
    
    n_epochs_before = len(epochs)
    epochs.drop_bad(REJECT_THRESHOLD)
    n_rejected = n_epochs_before - len(epochs)
    
    print(f'Epochs rejected: {n_rejected} / {n_epochs_before}')
    
    return epochs

In [ ]:
# Create epochs
epochs = make_epochs(subject_id)
epoch_filename = join(MEG_DIR, subject_id, f'{subject_id}_{EXPERIMENT_NAME}-ica-epo.fif')
epochs.save(epoch_filename, overwrite=True)

Now to making source models of brain activity. \
You can stop here and use the processed sensor data for analysis in sensor space. \
You should plot evoked data before moving to source space.

In [ ]:
# Parameters for making STCs
SNR = 3.0
METHOD = "dSPM"
FIXED_ORIENTATION = False # False, signed data and does not run mne.convert_forward_solution(fwd, surf_ori=True)
N_JOBS = 4
LAMBDA2 = 1.0 / SNR ** 2.0

In [ ]:
# Helper functions
def get_evokeds(subject_id, epochs):
    evokeds = {}
    conditions = list(epochs.metadata.condition.unique())
    
    for condition in conditions:
        condition_epochs = epochs[epochs.metadata["condition"] == condition]
        evoked = condition_epochs.average()
        evokeds[condition] = evoked
    
    return evokeds

def get_source_space(subject_id, src_fname, force_new=False):
    if not os.path.isfile(src_fname) or force_new:
        print(f'Creating source space for {subject_id}...')
        src = mne.setup_source_space(
            subject=subject_id, 
            spacing='ico4', 
            subjects_dir=SUBJECTS_DIR
        )
        src.save(src_fname, overwrite=True)
    else:
        src = mne.read_source_spaces(src_fname)
    return src

def get_forward_solution(subject_id, info, src, trans, bem_fname, force_new=False):
    fwd_path = os.path.join(MEG_DIR, subject_id)
    fwd_fname = os.path.join(fwd_path, f'{subject_id}-fwd.fif')
    
    os.makedirs(fwd_path, exist_ok=True)
    
    if not os.path.isfile(fwd_fname) or force_new:
        fwd = mne.make_forward_solution(
            info=info, trans=trans, src=src, 
            bem=bem_fname, ignore_ref=True
        )
        mne.write_forward_solution(fwd_fname, fwd, overwrite=True)
    else:
        fwd = mne.read_forward_solution(fwd_fname)
    return fwd

def get_covariance_matrix(subject_id, epochs, force_new=False):
    cov_fname = os.path.join(MEG_DIR, subject_id, f'{subject_id}-cov.fif')
    
    if not os.path.isfile(cov_fname) or force_new:
        cov = mne.compute_covariance(
            epochs, tmin=-0.1, tmax=0,
            method=['shrunk', 'diagonal_fixed', 'empirical']
        )
        cov.save(cov_fname, overwrite=True)
    else:
        cov = mne.read_cov(cov_fname)
    return cov

def create_and_save_stcs(subject_id, evokeds, inv, lambda2):
    for condition, evoked in evokeds.items():
        stc = mne.minimum_norm.apply_inverse(
            evoked, inv, lambda2=lambda2, method=METHOD
        )
        
        morph = mne.compute_source_morph(
            stc, subject_from=subject_id, subject_to='fsaverage',
            subjects_dir=SUBJECTS_DIR, spacing=4
        )
        stc_fsaverage = morph.apply(stc)
        
        stc_path = os.path.join(STC_DIR, condition)
        os.makedirs(stc_path, exist_ok=True)
        stc_filename = os.path.join(stc_path, f'{subject_id}_{condition}_dSPM')
        stc_fsaverage.save(stc_filename, overwrite=True)

In [ ]:
# Main processing loop
for i, subject_id in enumerate(subjects):
    print(f"Processing subject ({i+1}/{len(subjects)}): {subject_id}")
    
    try:
        trans_fname = os.path.join(MEG_DIR, subject_id, f'{subject_id}-trans.fif')
        src_fname = os.path.join(SUBJECTS_DIR, subject_id, 'bem', f'{subject_id}-ico-4-src.fif')
        bem_fname = os.path.join(SUBJECTS_DIR, subject_id, 'bem', f'{subject_id}-inner_skull-bem-sol.fif')
        epoch_file = os.path.join(MEG_DIR, subject_id, f'{subject_id}_{EXPERIMENT_NAME}-ica-epo.fif')
        
        trans = mne.read_trans(trans_fname)
        epochs = mne.read_epochs(epoch_file)
        info = epochs.info
        
        src = get_source_space(subject_id, src_fname)
        evokeds = get_evokeds(subject_id, epochs)
        fwd = get_forward_solution(subject_id, info, src, trans, bem_fname)
        cov = get_covariance_matrix(subject_id, epochs)
        
        if FIXED_ORIENTATION:
            fwd = mne.convert_forward_solution(fwd, surf_ori=True)
        
        inv = mne.minimum_norm.make_inverse_operator(
            info, fwd, cov, depth=0.8, loose='auto', fixed=FIXED_ORIENTATION
        )
        
        create_and_save_stcs(subject_id, evokeds, inv, LAMBDA2)
        print(f"✓ Completed {subject_id}")
        
    except Exception as e:
        print(f"✗ Error with {subject_id}: {e}")